In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tabulate import tabulate
from plotly.subplots import make_subplots

In [3]:
branches = pd.read_csv('../data/branches.csv')

commits = pd.read_csv('../data/branches.csv')

issues_prs = pd.read_csv('../data/branches.csv')

metadata = pd.read_csv('../data/branches.csv')

jira_data = pd.read_csv('../data/branches.csv')

github_devs = pd.read_csv('../data/branches.csv')

# Analisando produtividade no Jira

In [ ]:
def analyze_jira_contributors(df):
    # Criar contagem de tarefas por assignee
    contributor_stats = df.groupby('assignee').agg({
        'issue_key': 'count',
        'priority': lambda x: (x == 'High').sum(),
        'status': lambda x: (x == 'Concluído').sum()
    }).reset_index()
    
    contributor_stats.columns = ['Contribuidor', 'Total de Tarefas', 'Tarefas Prioritárias', 'Tarefas Concluídas']
    
    # Remover possíveis valores nulos ou vazios
    contributor_stats = contributor_stats.dropna()
    
    # Ordenar por total de tarefas
    contributor_stats = contributor_stats.sort_values('Total de Tarefas', ascending=True)
    
    # Criar gráfico
    fig = px.bar(
        contributor_stats.tail(10),  # Pegar os top 10
        x=['Total de Tarefas', 'Tarefas Prioritárias', 'Tarefas Concluídas'],
        y='Contribuidor',
        orientation='h',
        title='Análise de Contribuições por Membro da Equipe',
        barmode='stack',  # Mudando para barras empilhadas
        height=600,  # Aumentando altura
        width=1000,  # Definindo largura
        template='plotly_dark',  # Tema mais moderno
        color_discrete_sequence=['#2ecc71', '#e74c3c', '#3498db']  # Cores personalizadas
    )
    
    # Personalizar layout
    fig.update_layout(
        title_x=0.5,  # Centralizar título
        title_font_size=24,
        xaxis_title='Número de Tarefas',
        yaxis_title='Contribuidor',
        legend_title='Tipo de Tarefa',
        showlegend=True,
        margin=dict(l=200, r=50, t=100, b=50),
        paper_bgcolor='rgba(0,0,0,0)',  # Fundo transparente
        plot_bgcolor='rgba(0,0,0,0)',   # Fundo do plot transparente
        font=dict(
            size=14,
            family="Arial"
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=14,
            font_family="Arial"
        )
    )
    
    # Adicionar linhas de grade apenas no eixo x
    fig.update_xaxes(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='rgba(128,128,128,0.5)'
    )
    
    # Personalizar hover
    fig.update_traces(
        hovertemplate="<b>%{y}</b><br>" +
                     "Quantidade: %{x}<br>" +
                     "<extra></extra>"
    )
    
    return fig

# Para usar:
fig = analyze_jira_contributors(jira_data)
fig.show()

# Analisando commits por dev

In [53]:
def analyze_github_commits(df_commits, df_devs):

    commits_analysis = df_commits.merge(
        df_devs[['id', 'name']],
        left_on='author_id',
        right_on='id',
        how='left'
    )
    
    # Agregar dados por autor
    author_stats = commits_analysis.groupby('name').agg({
        'sha': 'count',                    # Total de commits
        'dmm_unit_complexity': 'mean',     # Complexidade média
        'dmm_unit_size': 'mean',          # Tamanho médio
        'dmm_unit_interfacing': 'mean'     # Interfaceamento médio
    }).reset_index()
    
    # Arredondar valores DMM para 3 casas decimais
    for col in ['dmm_unit_complexity', 'dmm_unit_size', 'dmm_unit_interfacing']:
        author_stats[col] = author_stats[col].round(3)
    
    # Ordenar por número de commits
    author_stats = author_stats.sort_values('sha', ascending=True)
    
    # Criar figura com subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Commits por Desenvolvedor',
            'Métricas DMM por Desenvolvedor'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}]],
        horizontal_spacing=0.15
    )
    
    # 1. Gráfico de barras horizontal - Commits por desenvolvedor
    fig.add_trace(
        go.Bar(
            y=author_stats['name'],
            x=author_stats['sha'],
            orientation='h',
            name='Total de Commits',
            marker_color='#2ecc71',
            text=author_stats['sha'],
            textposition='auto',
            hovertemplate="<b>%{y}</b><br>" +
                         "Commits: %{x}<br>" +
                         "<extra></extra>"
        ),
        row=1, col=1
    )
    
    # 2. Gráfico de barras horizontais para métricas DMM
    fig.add_trace(
        go.Bar(
            y=author_stats['name'],
            x=author_stats['dmm_unit_size'],
            orientation='h',
            name='Tamanho',
            marker_color='#3498db',
            text=author_stats['dmm_unit_size'],
            textposition='auto',
            hovertemplate="<b>%{y}</b><br>" +
                         "Tamanho: %{x:.3f}<br>" +
                         "<extra></extra>"
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(
            y=author_stats['name'],
            x=author_stats['dmm_unit_complexity'],
            orientation='h',
            name='Complexidade',
            marker_color='#e74c3c',
            text=author_stats['dmm_unit_complexity'],
            textposition='auto',
            hovertemplate="<b>%{y}</b><br>" +
                         "Complexidade: %{x:.3f}<br>" +
                         "<extra></extra>"
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(
            y=author_stats['name'],
            x=author_stats['dmm_unit_interfacing'],
            orientation='h',
            name='Interfaceamento',
            marker_color='#f1c40f',
            text=author_stats['dmm_unit_interfacing'],
            textposition='auto',
            hovertemplate="<b>%{y}</b><br>" +
                         "Interfaceamento: %{x:.3f}<br>" +
                         "<extra></extra>"
        ),
        row=1, col=2
    )
    
    # Atualizar layout
    fig.update_layout(
        height=max(400, len(author_stats) * 50),  # Altura dinâmica baseada no número de desenvolvedores
        width=1400,
        showlegend=True,
        title={
            'text': 'Análise de Commits e Métricas DMM por Desenvolvedor',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 24}
        },
        template='plotly_dark',
        barmode='group',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bgcolor="rgba(0,0,0,0.5)",
            bordercolor="white",
            borderwidth=1
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=14,
            font_family="Arial"
        )
    )
    
    # Atualizar eixos e legendas
    fig.update_xaxes(title_text='Número de Commits', row=1, col=1, gridcolor='rgba(128,128,128,0.2)', zeroline=True, zerolinecolor='rgba(128,128,128,0.5)')
    fig.update_xaxes(title_text='Valor DMM', row=1, col=2, gridcolor='rgba(128,128,128,0.2)', zeroline=True, zerolinecolor='rgba(128,128,128,0.5)')
    fig.update_yaxes(title_text='Desenvolvedor', row=1, col=1, showgrid=False)
    fig.update_yaxes(title_text='', row=1, col=2, showgrid=False)
    
    # Ajustar margens
    fig.update_layout(margin=dict(l=20, r=20, t=100, b=20))
    
    return fig

# Para usar:
fig = analyze_github_commits(commits, github_devs)
fig.show()